# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [ ]:
import pandas as pd
import pymongo

In [ ]:
# client = pymongo.MongoClient()
# database = client['exercices']
# collection = database['youtube']

In [ ]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

## Question 0

### Nettoyer les données

In [ ]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])

df_youtube.rename(columns={'video_id':'_id'}, inplace=True)

df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')

l_d =df_youtube.to_dict(orient='records')

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)

for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongo")

db_yt = client.yt

collection_yt = db_yt['videos']

collection_yt.delete_many({})

collection_yt.insert_many(l_d)

## Question 1  

In [ ]:
videosOfApple = collection_yt.find({
    "channel_title": "Apple"
})
list(videosOfApple)

## Question 2

In [ ]:
numbersOfCategories = collection_yt.aggregate([
    { "$group": { "_id": "$category_id"}  },
    { "$group": { "_id": "category_id", "count": { "$sum": 1 } } } 
])
list(numbersOfCategories)

## Question 3

In [ ]:
collection_yt.aggregate([
    { "$project": {
           "tags": {"$split":["$tags", "|"]}
       } }, 
    { "$merge": "videos" }
]);


## Question 4

In [ ]:
numberOfVideos = 10
mostViewedVideos = collection_yt.find().sort('views', pymongo.DESCENDING).limit(numberOfVideos)
list(mostViewedVideos)

## Question 5

In [ ]:
averageViewsOfCategories = collection_yt.aggregate([{
    '$group':{
        '_id':"$category_name", 
        "count": {"$sum" : 1 },
        "average": {"$avg": "$views"}
    }
}]);
list(averageViewsOfCategories)

## Question 6 

In [ ]:
youtubeVideosWithGreatestAverageOfLikes = collection_yt.aggregate([{
    '$group':{
        '_id':"$channel_title",
        'averageOfLikes': {"$avg": "$likes"}
    }
}, {
    '$sort': {
        'averageOfLikes': -1
    }
}]);
list(youtubeVideosWithGreatestAverageOfLikes)